# IHT.jl tutorial

In this tutorial we explore some of the functionality of the IHT.jl package, which implements iterative hard threhsolding on floating point arrays.

IHT minimizes the residual sum of squares $\frac{1}{2} \| \boldsymbol{y} - \boldsymbol{X} \boldsymbol{\beta} \|$ for data matrix $\boldsymbol{X}$, response vector $\boldsymbol{y}$, and coefficient vector $\boldsymbol{\beta}$. If $\boldsymbol{\beta}$ is $k$-sparse, then we have a sparse regression problem.

Let us start by defining several simulation parameters:

In [1]:
addprocs(5) # for later parallel XV
using IHT
n = 5000
p = 23999
k = 10
s = 0.1
x_temp = randn(n,p)

5000x23999 Array{Float64,2}:
  0.920157     0.255005  -0.327352   …  -0.621671   -0.60151     -0.106313 
  0.395504    -0.102723  -0.0716819      0.965816   -0.196628     0.993377 
 -0.28422      1.32892    0.258232      -0.15159    -0.648796     0.76599  
 -0.430892     0.182859  -1.73484        0.269101   -1.45088     -0.0857599
 -1.47349     -0.388611  -1.41136       -0.0954367  -2.04859     -0.317166 
 -1.49638     -1.05493   -0.98462    …  -0.280058    0.410015    -1.04961  
  1.57259     -0.639496   0.214717       0.912603    3.44151     -0.849509 
 -1.32385      0.319633   1.59429       -1.65875     2.50363      0.426919 
  0.513201     0.564551   1.46493       -0.60994     0.121982     3.32438  
 -0.840922     0.734688   1.03843       -1.21892     0.148621     1.63371  
  1.7489       0.717936  -1.20321    …   0.68411     0.518235     1.9636   
 -0.806362     1.11894    0.500853       2.42465     1.49706     -0.251184 
  1.64923     -0.119484   1.37656       -0.0260431  -2.0126

Since we want our simulation to be reproducible, configure $\boldsymbol{\beta}$ with a fixed random seed:

In [2]:
srand(2016)
b = zeros(p)
b[1:k] = randn(k)
shuffle!(b)
bidx = find(b)

10-element Array{Int64,1}:
  5912
  6593
 10073
 13599
 14572
 14929
 18057
 18357
 23140
 23528

Now we make a noisy response $\boldsymbol{y}$:

In [3]:
y = x_temp*b + s*randn(n)

5000-element Array{Float64,1}:
  2.57     
 -1.02483  
 -3.1841   
  0.0554894
  2.22562  
 -0.745217 
  0.605257 
 -0.0122337
  1.33673  
 -3.4684   
 -4.25107  
 -4.11406  
 -6.93096  
  ⋮        
 -1.31403  
  0.775042 
 -0.445469 
 -2.31673  
 -1.59955  
  0.45805  
 -0.837979 
  1.77936  
 -0.62778  
  0.898485 
 -0.777918 
 -4.6423   

Next we configure a regression problem. In this case, we need a data matrix with a grand mean included:

In [4]:
x = zeros(n,p+1)
setindex!(x, x_temp, :, 1:p)
x[:,end] = 1.0
x

5000x24000 Array{Float64,2}:
  0.920157     0.255005  -0.327352   …  -0.60151     -0.106313   1.0
  0.395504    -0.102723  -0.0716819     -0.196628     0.993377   1.0
 -0.28422      1.32892    0.258232      -0.648796     0.76599    1.0
 -0.430892     0.182859  -1.73484       -1.45088     -0.0857599  1.0
 -1.47349     -0.388611  -1.41136       -2.04859     -0.317166   1.0
 -1.49638     -1.05493   -0.98462    …   0.410015    -1.04961    1.0
  1.57259     -0.639496   0.214717       3.44151     -0.849509   1.0
 -1.32385      0.319633   1.59429        2.50363      0.426919   1.0
  0.513201     0.564551   1.46493        0.121982     3.32438    1.0
 -0.840922     0.734688   1.03843        0.148621     1.63371    1.0
  1.7489       0.717936  -1.20321    …   0.518235     1.9636     1.0
 -0.806362     1.11894    0.500853       1.49706     -0.251184   1.0
  1.64923     -0.119484   1.37656       -2.01264     -0.960593   1.0
  ⋮                                  ⋱                              
 -0.7

Using `x`, run iterative hard thresholding to pull the best model of size `k`. The function call is `L0_reg`, which returns a `Dict{ASCIIString,Any}` with the following key-value pairs:

    - "time" => compute time
    - "iter" => iterations taken
    - "loss" => residual sum of squares at convergence
    - "beta" => beta vector at convergence

In [5]:
output = L0_reg(x,y,k);
bk = copy(output["beta"])
[b[bidx] bk[bidx]]

10x2 Array{Float64,2}:
 -0.938486  -0.93735 
 -1.05075   -1.05182 
  1.47077    1.47139 
 -0.139799  -0.141464
 -1.21187   -1.21198 
 -0.974419  -0.972236
 -1.10666   -1.10799 
  0.429356   0.428427
 -2.09573   -2.09758 
 -0.494523  -0.49493 

Observe that IHT returns all the correct nonzero coefficients. The coefficient values themselves are fairly close to their originals. We should expect this since `s = 0.1` is not very strong noise. Observe what happens when we increase the noise:

In [6]:
s = 10
y2 = x_temp*b + s*randn(n)
output2 = L0_reg(x, y2, k);
bk2 = copy(output2["beta"])
[b[bidx] bk[bidx] bk2[bidx]]

10x3 Array{Float64,2}:
 -0.938486  -0.93735   -1.14044 
 -1.05075   -1.05182   -0.779405
  1.47077    1.47139    1.40886 
 -0.139799  -0.141464   0.0     
 -1.21187   -1.21198   -0.963054
 -0.974419  -0.972236  -1.1041  
 -1.10666   -1.10799   -1.14329 
  0.429356   0.428427   0.0     
 -2.09573   -2.09758   -2.31173 
 -0.494523  -0.49493   -0.802226

## Regularization Paths and LASSO

The coefficient values are less accurate, and we lost three nonzeroes. Still, the model recovery performance of IHT is not bad. We still need some sort of benchmark. [GLMNet.jl](https://github.com/simonster/GLMNet.jl) offers one way to test this:

In [7]:
using GLMNet
srand(2016)
nmodels = 30
lassopath = glmnet(x, y2, dfmax=nmodels, nlambda=nmodels) # try to test only a few models

Least Squares GLMNet Solution Path (11 solutions for 24000 predictors in 47 passes):
11×3 DataFrames.DataFrame
│ Row │ df │ pct_dev   │ λ        │
├─────┼────┼───────────┼──────────┤
│ 1   │ 0  │ 0.0       │ 2.28262  │
│ 2   │ 1  │ 0.0123469 │ 1.94746  │
│ 3   │ 1  │ 0.0213341 │ 1.66151  │
│ 4   │ 1  │ 0.0278759 │ 1.41755  │
│ 5   │ 2  │ 0.0364219 │ 1.20941  │
│ 6   │ 5  │ 0.050224  │ 1.03183  │
│ 7   │ 6  │ 0.0644825 │ 0.880322 │
│ 8   │ 8  │ 0.0767626 │ 0.751062 │
│ 9   │ 8  │ 0.0874198 │ 0.640782 │
│ 10  │ 12 │ 0.0963946 │ 0.546695 │
│ 11  │ 33 │ 0.10967   │ 0.466422 │

We want the estimates of $\boldsymbol{\beta}$ with `k = 10` nonzeroes. In this example, the LASSO does not return the precise model size that we want (a pointed advantage of IHT!) but row 9 will give us a decent approximation.

In [8]:
betas = convert(Matrix{Float64}, lassopath.betas)
[bk2[bidx] betas[bidx,9]]

10x2 Array{Float64,2}:
 -1.14044   -0.48379 
 -0.779405  -0.146337
  1.40886    0.751371
  0.0        0.0     
 -0.963054  -0.337626
 -1.1041    -0.505814
 -1.14329   -0.525992
  0.0        0.0     
 -2.31173   -1.65493 
 -0.802226  -0.159854

LASSO selects the correct nonzeroes, but the coefficients are shrunk because LASSO is a shrinkage operator. Users should consider this fact when choosing a feature selection tool. In some scenarios, such as genome-wide association studies, the coefficient values are often quite small, and shrinkage can complicate accurate estimation of the statistical model.

LASSO is most efficient when used to compute a regularization path. IHT.jl provides `iht_path` to mimic this behavior.
Let us compute model sizes $1, 2, \ldots, 30$ with IHT:

In [9]:
colidx  = countnz(sub(betas, :, 9))
pathidx = collect(1:nmodels)
ihtpath = iht_path(x, y2, pathidx) # ihtpath is a sparse matrix
[bk2[bidx] full(ihtpath[bidx,colidx])]

10x2 Array{Float64,2}:
 -1.14044   -1.14763 
 -0.779405  -0.752326
  1.40886    1.40883 
  0.0        0.0     
 -0.963054  -0.971061
 -1.1041    -1.11848 
 -1.14329   -1.16075 
  0.0        0.0     
 -2.31173   -2.30384 
 -0.802226  -0.806393

## Crossvalidation

These exploratory efforts are admittedly not illuminating. In a realistic setting, we wouldn't know the correct model size `k = 10`. LASSO deals with this by crossvalidating the regularization path. IHT can do the same with `cv_iht`:

In [10]:
nfolds = 5
srand(2016)
cvlasso = glmnetcv(x, y2, dfmax=nmodels, nlambda=nmodels, nfolds=nfolds)
errors, b_cv_iht, bidx_cv_iht = cv_iht(x, y2, pathidx, nfolds);

Both IHT and LASSO refit their best model sizes. `cv_iht` returns the best $\boldsymbol{\beta}$ directly, but we need to extract it from LASSO:

In [11]:
b_cv_lasso = cvlasso.path.betas[:,indmin(cvlasso.meanloss)]
bidx_cv_lasso = find(b_cv_lasso)
display(bidx_cv_iht)
display(bidx_cv_lasso)

6-element Array{Int64,1}:
  5912
 10073
 14572
 14929
 18057
 23140

12-element Array{Int64,1}:
  4796
  5912
  6593
 10073
 13550
 14572
 14929
 18057
 18357
 21358
 23140
 23528

LASSO selects far too many features. IHT selects a more parsimonious set, though it underestimates the true model size. Take a peek at the estimated best $\boldsymbol{\beta}$ values compared to the true ones:

In [12]:
bfull_cv = zeros(size(b))
bfull_cv[bidx_cv_iht] = b_cv_iht
[b[bidx] bfull_cv[bidx] b_cv_lasso[bidx]]

10x3 Array{Float64,2}:
 -0.938486  -1.13444   -0.580837 
 -1.05075    0.0       -0.238076 
  1.47077    1.42045    0.847557 
 -0.139799   0.0        0.0      
 -1.21187   -0.993201  -0.430349 
 -0.974419  -1.1232    -0.594152 
 -1.10666   -1.15451   -0.61679  
  0.429356   0.0        0.0177542
 -2.09573   -2.30607   -1.7504   
 -0.494523   0.0       -0.254586 

In crossvalidating the best model size, IHT loses the smallest components but estimates the remaining coefficients reasonably well. LASSO grabs more of the true model but also pulls in a lot of garbage.